<a href="https://colab.research.google.com/github/DangKhoa-VT2-PTIT/DCGAN_aug_face/blob/main/GAN_face_augmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Tăng cường dữ liệu với GAN

In [ ]:
!python --version

!pip install tensorflow
!pip install matplotlib

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, Dense, BatchNormalization, Flatten, InputLayer, LeakyReLU, Conv2DTranspose, Reshape, Dropout
from tensorflow.keras import Model

import matplotlib.pyplot as plt

import os
import time

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
TRAIN_DIR = "/content/drive/MyDrive/images_resize/ngan"
IMAGE_SIZE = (96,80)
BATCH_SIZE = 64

# Chargement des données d'entraînement
train_data = load_and_preprocess_dataset(TRAIN_DIR, IMAGE_SIZE, BATCH_SIZE, shuffle=True)

Found 700 files.


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
import os
from tensorflow.keras.preprocessing.image import load_img, img_to_array, save_img
import matplotlib.pyplot as plt

# Generator Model
def build_generator(latent_dim=256):
    model = tf.keras.Sequential([
        layers.Dense(4 * 4 * 2048, use_bias=False, input_shape=(latent_dim,)),
        layers.BatchNormalization(),
        layers.ReLU(),
        layers.Reshape((4, 4, 2048)),

        layers.Conv2DTranspose(1024, kernel_size=4, strides=2, padding='same', use_bias=False),
        layers.BatchNormalization(),
        layers.ReLU(),

        layers.Conv2DTranspose(512, kernel_size=4, strides=2, padding='same', use_bias=False),
        layers.BatchNormalization(),
        layers.ReLU(),

        layers.Conv2DTranspose(256, kernel_size=4, strides=2, padding='same', use_bias=False),
        layers.BatchNormalization(),
        layers.ReLU(),

        layers.Conv2DTranspose(128, kernel_size=4, strides=2, padding='same', use_bias=False),
        layers.BatchNormalization(),
        layers.ReLU(),

        layers.Conv2DTranspose(3, kernel_size=4, strides=2, padding='same', use_bias=False, activation='tanh')
    ])
    return model

# Discriminator Model
def build_discriminator():
    model = tf.keras.Sequential([
        layers.Conv2D(128, kernel_size=4, strides=2, padding='same', input_shape=(128, 128, 3)),
        layers.LeakyReLU(0.2),

        layers.Conv2D(256, kernel_size=4, strides=2, padding='same'),
        layers.BatchNormalization(),
        layers.LeakyReLU(0.2),

        layers.Conv2D(512, kernel_size=4, strides=2, padding='same'),
        layers.BatchNormalization(),
        layers.LeakyReLU(0.2),

        layers.Conv2D(1024, kernel_size=4, strides=2, padding='same'),
        layers.BatchNormalization(),
        layers.LeakyReLU(0.2),

        layers.Flatten(),
        layers.Dense(1, activation='sigmoid')
    ])
    return model

# Resize input image
def resize_image(image_path, target_size=(128, 128)):
    image = load_img(image_path)
    image = image.resize(target_size)
    image = img_to_array(image)
    image = (image - 127.5) / 127.5  # Normalize to [-1, 1]
    return np.expand_dims(image, axis=0)

# Save generated images
def save_generated_images(generated_images, output_dir):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    for i, img in enumerate(generated_images):
        img = (img * 127.5 + 127.5).astype(np.uint8)
        save_img(f"{output_dir}/generated_{i + 1}.png", img)

# Plot generated images
def plot_generated_images(generator, latent_dim, num_images=16):
    noise = np.random.normal(0, 1, (num_images, latent_dim))
    generated_images = generator.predict(noise)
    generated_images = (generated_images + 1) / 2.0  # Rescale to [0, 1]

    plt.figure(figsize=(8, 8))
    for i in range(num_images):
        # plt.subplot(4, 4, i + 1)
        plt.imshow(generated_images[i])
        plt.axis('off')
    plt.show()

# Training function
def train(generator, discriminator, dataset, latent_dim, epochs=10000, batch_size=64):
    cross_entropy = tf.keras.losses.BinaryCrossentropy()

    generator_optimizer = tf.keras.optimizers.Adam(1e-4)
    discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

    @tf.function
    def train_step(real_images):
        noise = tf.random.normal([batch_size, latent_dim])

        with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
            generated_images = generator(noise, training=True)

            real_output = discriminator(real_images, training=True)
            fake_output = discriminator(generated_images, training=True)

            gen_loss = cross_entropy(tf.ones_like(fake_output), fake_output)
            real_loss = cross_entropy(tf.ones_like(real_output), real_output)
            fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
            disc_loss = real_loss + fake_loss

        gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
        gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

        generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
        discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

        return gen_loss, disc_loss

    for epoch in range(epochs):
        for image_batch in dataset:
            gen_loss, disc_loss = train_step(image_batch)

        print(f"Epoch {epoch + 1}, Gen Loss: {gen_loss.numpy()}, Disc Loss: {disc_loss.numpy()}")

        # Plot generated images every 10 epochs
        if (epoch + 1) % 10 == 0:
            plot_generated_images(generator, latent_dim)

# Generate 100 images from a single latent vector
def generate_images(generator, latent_dim, num_images=100):
    noise = np.random.normal(0, 1, (num_images, latent_dim))
    generated_images = generator.predict(noise)
    return (generated_images + 1) / 2.0  # Rescale to [0, 1]

# Main execution
if __name__ == "__main__":
    # Paths
    image_path = "/content/drive/MyDrive/images_face/khoa/face_image_1_front.jpg"
    output_dir = "/content/drive/MyDrive/output_images/khoa/front"

    # Load and resize image (if needed)
    # Remove the batch dimension from the input image
    input_image = resize_image(image_path)
    input_image = np.squeeze(input_image, axis=0)

    # Build generator and discriminator
    latent_dim = 256
    generator = build_generator(latent_dim)
    discriminator = build_discriminator()

    # Prepare dataset
    dataset = tf.data.Dataset.from_tensor_slices([input_image]).repeat(100).batch(16)

    # Train GAN
    train(generator, discriminator, dataset, latent_dim, epochs=200, batch_size=16)

    # Generate images
    generated_images = generate_images(generator, latent_dim, num_images=100)

    # Save generated images
    save_generated_images(generated_images, output_dir)

    print("Generated images saved.")


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
import os
from tensorflow.keras.preprocessing.image import load_img, img_to_array, save_img
import matplotlib.pyplot as plt

# Generator Model
def build_generator(latent_dim=256):
    model = tf.keras.Sequential([
        layers.Dense(4 * 4 * 2048, use_bias=False, input_shape=(latent_dim,)),
        layers.BatchNormalization(),
        layers.ReLU(),
        layers.Reshape((4, 4, 2048)),

        layers.Conv2DTranspose(1024, kernel_size=4, strides=2, padding='same', use_bias=False),
        layers.BatchNormalization(),
        layers.ReLU(),

        layers.Conv2DTranspose(512, kernel_size=4, strides=2, padding='same', use_bias=False),
        layers.BatchNormalization(),
        layers.ReLU(),

        layers.Conv2DTranspose(256, kernel_size=4, strides=2, padding='same', use_bias=False),
        layers.BatchNormalization(),
        layers.ReLU(),

        layers.Conv2DTranspose(128, kernel_size=4, strides=2, padding='same', use_bias=False),
        layers.BatchNormalization(),
        layers.ReLU(),

        layers.Conv2DTranspose(3, kernel_size=4, strides=2, padding='same', use_bias=False, activation='tanh')
    ])
    return model

# Discriminator Model
def build_discriminator():
    model = tf.keras.Sequential([
        layers.Conv2D(128, kernel_size=4, strides=2, padding='same', input_shape=(128, 128, 3)),
        layers.LeakyReLU(0.2),

        layers.Conv2D(256, kernel_size=4, strides=2, padding='same'),
        layers.BatchNormalization(),
        layers.LeakyReLU(0.2),

        layers.Conv2D(512, kernel_size=4, strides=2, padding='same'),
        layers.BatchNormalization(),
        layers.LeakyReLU(0.2),

        layers.Conv2D(1024, kernel_size=4, strides=2, padding='same'),
        layers.BatchNormalization(),
        layers.LeakyReLU(0.2),

        layers.Flatten(),
        layers.Dense(1, activation='sigmoid')
    ])
    return model

# Resize input image
def resize_image(image_path, target_size=(128, 128)):
    image = load_img(image_path)
    image = image.resize(target_size)
    image = img_to_array(image)
    image = (image - 127.5) / 127.5  # Normalize to [-1, 1]
    return np.expand_dims(image, axis=0)

# Save generated images
def save_generated_images(generated_images, output_dir):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    for i, img in enumerate(generated_images):
        img = (img * 127.5 + 127.5).astype(np.uint8)
        save_img(f"{output_dir}/generated_{i + 1}.png", img)

# Plot generated images
def plot_generated_images(generator, latent_dim, num_images=16):
    noise = np.random.normal(0, 1, (num_images, latent_dim))
    generated_images = generator.predict(noise)
    generated_images = (generated_images + 1) / 2.0  # Rescale to [0, 1]

    plt.figure(figsize=(8, 8))
    for i in range(num_images):
        # plt.subplot(4, 4, i + 1)
        plt.imshow(generated_images[i])
        plt.axis('off')
    plt.show()

# Training function
def train(generator, discriminator, dataset, latent_dim, epochs=10000, batch_size=64):
    cross_entropy = tf.keras.losses.BinaryCrossentropy()

    generator_optimizer = tf.keras.optimizers.Adam(1e-4)
    discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

    @tf.function
    def train_step(real_images):
        noise = tf.random.normal([batch_size, latent_dim])

        with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
            generated_images = generator(noise, training=True)

            real_output = discriminator(real_images, training=True)
            fake_output = discriminator(generated_images, training=True)

            gen_loss = cross_entropy(tf.ones_like(fake_output), fake_output)
            real_loss = cross_entropy(tf.ones_like(real_output), real_output)
            fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
            disc_loss = real_loss + fake_loss

        gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
        gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

        generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
        discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

        return gen_loss, disc_loss

    for epoch in range(epochs):
        for image_batch in dataset:
            gen_loss, disc_loss = train_step(image_batch)

        print(f"Epoch {epoch + 1}, Gen Loss: {gen_loss.numpy()}, Disc Loss: {disc_loss.numpy()}")

        # Plot generated images every 10 epochs
        if (epoch + 1) % 10 == 0:
            plot_generated_images(generator, latent_dim)

# Generate 100 images from a single latent vector
def generate_images(generator, latent_dim, num_images=100):
    noise = np.random.normal(0, 1, (num_images, latent_dim))
    generated_images = generator.predict(noise)
    return (generated_images + 1) / 2.0  # Rescale to [0, 1]

# Main execution
if __name__ == "__main__":
    # Base paths
    base_image_path = "/content/drive/MyDrive/images_face/phu"
    base_output_dir = "/content/drive/MyDrive/output_images/phu"

    # List of image filenames
    image_files = [
        "face_image_1_front.jpg",
        "face_image_2_left.jpg",
        "face_image_3_right.jpg",
        "face_image_4_top.jpg",
        "face_image_5_bottom.jpg"
    ]

    latent_dim = 256
    for image_file in image_files:
        image_path = os.path.join(base_image_path, image_file)
        output_dir = os.path.join(base_output_dir, os.path.splitext(image_file)[0].split('_')[-1])
        print(os.path.splitext(image_file)[0].split('_')[-1])

        # Load and resize image (if needed)
        input_image = resize_image(image_path)
        input_image = np.squeeze(input_image, axis=0)

        # Build generator and discriminator
        generator = build_generator(latent_dim)
        discriminator = build_discriminator()

        # Prepare dataset
        dataset = tf.data.Dataset.from_tensor_slices([input_image]).repeat(100).batch(16)

        # Train GAN
        train(generator, discriminator, dataset, latent_dim, epochs=200, batch_size=16)

        # Generate images
        generated_images = generate_images(generator, latent_dim, num_images=100)

        # Save generated images
        save_generated_images(generated_images, output_dir)

        print(f"Generated images saved for {image_file}.")


front


ResourceExhaustedError: {{function_node __wrapped__StatelessRandomUniformV2_device_/job:localhost/replica:0/task:0/device:GPU:0}} OOM when allocating tensor with shape[4,4,1024,2048] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:StatelessRandomUniformV2] name: 